In [ ]:
"""FINAL version of calculating Splicing Efficiency.


JunctionSeq gives weird SJ coordinates: as start it sets last exonic base, and as end it sets last intronic base.
Whereas STAR aligner gives coordinates for intronic bases only.
Example for YGL030W/RPL30 gene: 
    STAR SJ: VII:439094-439323
    JunctionSeq SJ: VII:439093-439323  

This method uses coordinates generated by JunctionSeq. 

To get read counts we use coordinates for the last base of exon_1 (to extract exon-intron read counts at 5'SS) 
and first base of exon_2 (to extract intron-exon read counts at 3'SS)"""

import pandas as pd

def exe(cmd):
    import subprocess
    o = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out = o.stdout.read().strip().decode()
    err = o.stderr.read().strip().decode()
    return out, err

def spleff(decoder, samplist, bampath, outfolder, verbose=False): 

    """Calculation of splicing effciency.
    
    --> This method takes into account that when exon_1 is very short (<30 nt), 'p' value equals 0, 
        and so 'total' = EE + EI3.
        For novel junctions, which are upstream start of exon_1, intron length is gonna get negative value.
    --> Importantly, gene orientation is taken into account too!     
    
    Attributes:

        decoder (string) : path to a file generated by JunctionSeq; this file name ends with "bySampleallGenes.results.txt";
            particularly you need columns named: 'featureID', 'geneID', 'chr', 'start', 'end', 
            and 'CDSstart', 'CDSend', 'Strand' (these you need to add to a table by yourself BEFORE running this program! save it as excel file);
            'Strand' is needed to account for gene orientation (which values are junction 'start' and which are 'stop')
            featureID needs to look like this: YGL030W:J003
        
        samplist (list) : list of sample names, e.g. "MEX_WT_7", "MEX_WT_8", "MEX_MUT_10", "MEX_MUT_11";
            names in this list are simply a part of each BAM file name, e.g. from file name "MEX_WT_7Aligned.sortedByCoord.out.bam" use "MEX_WT_7"
        
        bampath (string): part of the path to each BAM file shared between all BAM files to be used, 
            e.g. "/Volumes/agata/Desktop/RNAseq/output_copied/[...]"
            Important for other purposes --> BAM file name has to end with: "*Aligned.sortedByCoord.out.bam"
        
        outfolder (string): path to a folder where you store resulting excel files  
        
    Example::
    
         decoderMex = "/Users/agata/Desktop/RNAseq_results/opt/QoRTs_JunctionSeq/2nd_approach/Splicing_Efficiency/decoder_MEX67_exStartStopStrand.xls"
         samplistMex = ["MEX_WT_7", "MEX_WT_8", "MEX_WT_9", "MEX_MUT_10", "MEX_MUT_11", "MEX_MUT_12"]
         bampath = "/Volumes/agata/Desktop/RNAseq/output_copied/"
         outfolder = "/Users/agata/Desktop/RNAseq_results/opt/QoRTs_JunctionSeq/2nd_approach/Splicing_Efficiency/"
    
    """
    
    df = pd.read_excel(decoder)
    for sample in samplist:  
        dfout = pd.DataFrame()
        bam = bampath + sample + "Aligned.sortedByCoord.out.bam"
        counter = 0
        for index, row in df.iterrows():
            f = row['featureID']
            gene = row['geneID']
            ch = row['chr']
            start = row['start']
            end = row['end']
            CDSstart = row['CDSstart']
            CDSend = row['CDSend']
            strand = row['Strand']
            gene_name, element = f.split(':')
            if element.startswith('J') or element.startswith('N'):
                if strand == "+":
                    midintron = start + int((end - start)/2)
                    cmd = "/usr/local/bin/samtools view " + bam + " " + ch + ":" + str(midintron) + "-" + str(midintron) + "| cut -f 6 > /tmp/SE" 
                    out, err = exe(cmd)
                    cmd = "cat /tmp/SE | grep 'N' | wc -l"
                    EE_ct, err = exe(cmd)
                    EE_ct = int(EE_ct)

                    cmd = "/usr/local/bin/samtools view " + bam + " " + ch + ":" + str(start) + "-" + str(start) + "| cut -f 6 > /tmp/SE" 
                    out, err = exe(cmd)
                    cmd = "cat /tmp/SE | grep -v 'N' | wc -l"
                    EI5_ct, err = exe(cmd)
                    EI5_ct = int(EI5_ct)

                    cmd = "/usr/local/bin/samtools view " + bam + " " + ch + ":" + str(int(end)+1) + "-" + str(int(end)+1) + "| cut -f 6 > /tmp/SE" 
                    out, err = exe(cmd)
                    cmd = "cat /tmp/SE | grep -v 'N' | wc -l"
                    EI3_ct, err = exe(cmd)
                    EI3_ct = int(EI3_ct)
                
                    ex1_len = start - CDSstart
                    if ex1_len < 30:
                        p = 0
                        total = EE_ct + EI3_ct
                        se = round(EE_ct / total, 4)
                    else:    
                        if (EI5_ct + EI3_ct) != 0 and EE_ct != 0:
                            p = round(EI5_ct / (EI5_ct + EI3_ct), 6)
                            total = round(EE_ct + (p * EI5_ct) + (1 - p) * EI3_ct, 2)
                            se = round(EE_ct / total, 4)
                        elif (EI5_ct + EI3_ct) == 0 and EE_ct != 0:
                            p = 0
                            total = EE_ct
                            se = 1
                        elif (EI5_ct + EI3_ct) != 0 and EE_ct == 0:
                            p = round(EI5_ct / (EI5_ct + EI3_ct), 6)
                            total = round(EE_ct + (p * EI5_ct) + (1 - p) * EI3_ct, 2)
                            se = 0
                        else:
                            p = "N/A"
                            total = "N/A"
                            se = "N/A"
                if strand == "-":
                    midintron = start + int((end - start)/2)
                    cmd = "/usr/local/bin/samtools view " + bam + " " + ch + ":" + str(midintron) + "-" + str(midintron) + "| cut -f 6 > /tmp/SE" 
                    out, err = exe(cmd)
                    cmd = "cat /tmp/SE | grep 'N' | wc -l"
                    EE_ct, err = exe(cmd)
                    EE_ct = int(EE_ct)

                    cmd = "/usr/local/bin/samtools view " + bam + " " + ch + ":" + str(start) + "-" + str(start) + "| cut -f 6 > /tmp/SE" 
                    out, err = exe(cmd)
                    cmd = "cat /tmp/SE | grep -v 'N' | wc -l"
                    EI3_ct, err = exe(cmd)
                    EI3_ct = int(EI3_ct)

                    cmd = "/usr/local/bin/samtools view " + bam + " " + ch + ":" + str(int(end)+1) + "-" + str(int(end)+1) + "| cut -f 6 > /tmp/SE" 
                    out, err = exe(cmd)
                    cmd = "cat /tmp/SE | grep -v 'N' | wc -l"
                    EI5_ct, err = exe(cmd)
                    EI5_ct = int(EI5_ct)
                    
                    ex1_len = CDSend - end
                    if ex1_len < 30:
                        p = 0
                        total = EE_ct + EI3_ct
                        se = round(EE_ct / total, 4)
                    else:    
                        if (EI5_ct + EI3_ct) != 0 and EE_ct != 0:
                            p = round(EI5_ct / (EI5_ct + EI3_ct), 6)
                            total = round(EE_ct + (p * EI5_ct) + (1 - p) * EI3_ct, 2)
                            se = round(EE_ct / total, 4)
                        elif (EI5_ct + EI3_ct) == 0 and EE_ct != 0:
                            p = 0
                            total = EE_ct
                            se = 1
                        elif (EI5_ct + EI3_ct) != 0 and EE_ct == 0:
                            p = round(EI5_ct / (EI5_ct + EI3_ct), 6)
                            total = round(EE_ct + (p * EI5_ct) + (1 - p) * EI3_ct, 2)
                            se = 0
                        else:
                            p = "N/A"
                            total = "N/A"
                            se = "N/A"

                counter = counter + 1
                
                print(gene, f, ch, start, end, EI5_ct, EI3_ct, EE_ct, p, total, ex1_len, se)

                dfout = dfout.append(pd.DataFrame({'geneID': gene,
                                                   'chr': ch,
                                                   'strand' : strand,
                                                   'featureID': f,
                                                   'I_5p': start,
                                                   'I_3p': end,
                                                   'ex1_length_' + sample : ex1_len,
                                                   'EI5_rawCt_' + sample : EI5_ct,
                                                   'EI3_rawCt_' + sample : EI3_ct,
                                                   'EE_rawCt_' + sample : EE_ct,
                                                   'p_' + sample : p,
                                                   'total_' + sample : total,
                                                   'SE_' + sample : se,
                                                  }, index=[counter]))
        
        
        outpath = outfolder + "SE_" + sample + '_p.And.ExLen.xls'
        if verbose: 
            print('outpath:', outpath)
        dfout.to_excel(outpath)


    
decoderMex = "/Users/agata/Desktop/RNAseq_results/opt/QoRTs_JunctionSeq/2nd_approach/Splicing_Efficiency/decoder_MEX67_exStartStopStrand.xls"
samplistMex = ["MEX_WT_7", "MEX_WT_8", "MEX_WT_9", "MEX_MUT_10", "MEX_MUT_11", "MEX_MUT_12"]

decoderNpl3 = "/Users/agata/Desktop/RNAseq_results/opt/QoRTs_JunctionSeq/2nd_approach/Splicing_Efficiency/decoder_NPL3_exStartStopStrand.xls"
samplistNpl3 = ["N3_WT_1", "N3_WT_2", "N3_WT_3", "DL_4", "DL_5", "DL_6"]

decoder305 = "/Users/agata/Desktop/RNAseq_results/opt/QoRTs_JunctionSeq/2nd_approach/Splicing_Efficiency/decoder_305_exStartStopStrand.xls"
samplist305 = ["MTR10_WT_13", "MTR10_WT_14", "MTR10_WT_15", "305_16", "305_17", "305_18"]

decoder406 = "/Users/agata/Desktop/RNAseq_results/opt/QoRTs_JunctionSeq/2nd_approach/Splicing_Efficiency/decoder_406_exStartStopStrand.xls"
samplist406 = ["406_19", "406_20", "406_21"]

bampath = "/Volumes/agata/Desktop/RNAseq/output_copied/"

outfolder = '/Users/agata/Desktop/RNAseq_results/opt/QoRTs_JunctionSeq/2nd_approach/Splicing_Efficiency/SE_p-calc/p-calc_ex1len/'

spleff(decoderMex, samplistMex, bampath, outfolder)
spleff(decoderNpl3, samplistNpl3, bampath, outfolder)
spleff(decoder305, samplist305, bampath, outfolder)
spleff(decoder406, samplist406, bampath, outfolder)

